In [27]:
# %cd ..
%cd /home/janneke/Documents/Master/Machine_Learning_in_Practice/HMS/MLiP_group_10_task1_HMS/

/home/janneke/Documents/Master/Machine_Learning_in_Practice/HMS/MLiP_group_10_task1_HMS


In [28]:
from copy import deepcopy

import pandas as pd
import numpy as np
import pickle
from logging import getLogger, basicConfig, INFO
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor

from generics import Generics
from datasets.raw_data_loader import CustomRawDataset
from datasets.data_loader import CustomDataset
from datasets.data_loader_configs import BaseDataConfig
from utils.evaluation_utils import score_kl_divergence
from utils.inference_utils import create_submission

In [29]:
basicConfig(level=INFO)
logger = getLogger('main')
config = BaseDataConfig()

In [30]:
train_subset_sample_count = 0
test_dataset = CustomRawDataset(config, mode="test", cache=True)
test_dataset.print_summary()

2024-02-17 13:45:06,903 - data_loader.log - INFO - Loading dataset from cache: ./data/cache/CustomRawDataset_0_test.npz
2024-02-17 13:45:06,903 - data_loader.log - INFO - Loading dataset from cache: ./data/cache/CustomRawDataset_0_test.npz
2024-02-17 13:45:06,903 - data_loader.log - INFO - Loading dataset from cache: ./data/cache/CustomRawDataset_0_test.npz
2024-02-17 13:45:06,903 - data_loader.log - INFO - Loading dataset from cache: ./data/cache/CustomRawDataset_0_test.npz
INFO:data_loader.log:Loading dataset from cache: ./data/cache/CustomRawDataset_0_test.npz


Dataset Summary:
Mode: test
Total Samples: 1
Unique Patients: 1
Unique EEGs: 1
Unique Spectrograms: 1
Probabilities Loaded: 1
Features Loaded: 1

Configuration Summary:
+---------------------+--------------------------------------------------------------------------------------------------------------------------------------+
| Configuration       | Value                                                                                                                                |
+---------------------+--------------------------------------------------------------------------------------------------------------------------------------+
| BATCH_SIZE_TEST     | 16                                                                                                                                   |
| BATCH_SIZE_TRAIN    | 64                                                                                                                                   |
| BATCH_SIZE_VAL      | 16          

In [32]:
x_test = test_dataset.features_per_sample
print(x_test.shape)

(1, 187)


In [33]:
with open(f"checkpoints/other_models/ensemble_one_model_per_target_{train_subset_sample_count}.pickle", "rb") as pickle_file:
	models = pickle.load(pickle_file)

Let multiple separate models regress on one label each

In [36]:
y_pred = np.zeros((x_test.shape[0], len(models)))  # shape: num_samles, num_labels

for i, (lbl, model) in enumerate(models.items()):
	y_pred_group = model.predict(x_test)
	y_pred[:,i] = y_pred_group

y_pred[y_pred < 0] = 0
y_pred_probabilities = y_pred / np.sum(y_pred, axis=1)[:,None]

GradientBoostingRegressor(learning_rate=1.0, max_depth=1, random_state=0)
GradientBoostingRegressor(learning_rate=1.0, max_depth=1, random_state=0)
GradientBoostingRegressor(learning_rate=1.0, max_depth=1, random_state=0)
GradientBoostingRegressor(learning_rate=1.0, max_depth=1, random_state=0)
GradientBoostingRegressor(learning_rate=1.0, max_depth=1, random_state=0)
GradientBoostingRegressor(learning_rate=1.0, max_depth=1, random_state=0)


In [37]:
submission_df = create_submission(test_dataset.main_df, y_pred_probabilities, Generics.LABEL_COLS, 'submission.csv')

Submission shape: (1, 7)
